In [1]:
import PySpin
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time

In [2]:
system = PySpin.System.GetInstance()
version = system.GetLibraryVersion()
print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
cam_list = system.GetCameras()

num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)

Library version: 1.20.0.15
Number of cameras detected: 1


In [3]:
for i, cam in enumerate(cam_list):
    nodemap = cam.GetTLDeviceNodeMap()
    node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))
    if PySpin.IsAvailable(node_device_information) and PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if PySpin.IsReadable(node_feature) else 'Node not readable'))

    else:
        print('Device control information not available.')

DeviceID: 17278795
DeviceSerialNumber: 17278795
DeviceVendorName: Point Grey Research
DeviceModelName: Flea3 FL3-U3-13Y3M
DeviceType: U3V
DeviceDisplayName: Point Grey Research Flea3 FL3-U3-13Y3M
DeviceAccessStatus: ReadWrite
DeviceVersion: FW:v2.7.3.00 FPGA:v2.02
DeviceDriverVersion: PGRUSBCam.sys : 2.7.3.235
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_3300&MI_00\7&2DB08C72&0&0000
DeviceCurrentSpeed: SuperSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1


In [4]:
for i, cam in enumerate(cam_list):
    # Initialize camera
    cam.Init()

In [5]:
for i, cam in enumerate(cam_list):

    # Set acquisition mode to continuous
    node_acquisition_mode = PySpin.CEnumerationPtr(cam.GetNodeMap().GetNode('AcquisitionMode'))
    if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
        print('Unable to set acquisition mode to continuous (node retrieval; camera %d). Aborting... \n' % i)
        # return False

    node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
    if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
            node_acquisition_mode_continuous):
        print('Unable to set acquisition mode to continuous (entry \'continuous\' retrieval %d). \
        Aborting... \n' % i)
        # return False

    acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

    node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

    print('Camera %d acquisition mode set to continuous...' % i)
    # print('Camera %d started acquiring images...' % i)

    # Retrieve device serial number for filename0
    node_device_serial_number = PySpin.CStringPtr(cam.GetTLDeviceNodeMap().GetNode('DeviceSerialNumber'))

    if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
        device_serial_number = node_device_serial_number.GetValue()
        print('Camera %d serial number set to %s...' % (i, device_serial_number))
    
    
    if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
        print('Unable to disable automatic exposure. Aborting...')
        raise ValueError
    cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
    print('Automatic exposure disabled...')
    
    # Ensure desired exposure time does not exceed the maximum
    exposure_time_to_set = 10 * 1000.0
    exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
    cam.ExposureTime.SetValue(exposure_time_to_set)
    
    cam.BinningVertical.SetValue(1)
    # cam.BinningHorizontal.SetValue(1)

Camera 0 acquisition mode set to continuous...
Camera 0 serial number set to 17278795...
Automatic exposure disabled...


In [6]:
cam = cam_list[0]

# Acquire!

In [68]:
# Begin acquiring images
cam.BeginAcquisition()
cv2.namedWindow('pointgrey', cv2.WINDOW_NORMAL)
should_continue = True
font = cv2.FONT_HERSHEY_SIMPLEX
while should_continue:
    image_result = cam.GetNextImage()
    if image_result.IsIncomplete():
        print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
        continue
    else:
        pass
    
    image_converted = image_result.Convert(PySpin.PixelFormat_Mono8, PySpin.HQ_LINEAR)
    image_data = image_converted.GetNDArray()
    
    timestamp = image_result.GetTimeStamp()
    image_result.Release()
    
    h, w= image_data.shape
    out = cv2.cvtColor(image_data, cv2.COLOR_GRAY2RGB)
    t = float(timestamp)*1e-9
    string = '%.9f' % t
    cv2.putText(out,string,(20,h-20), font, 1,(0,0,255),2,cv2.LINE_AA)
    
    cv2.imshow('pointgrey', out)
    # should_continue = True
    key = cv2.waitKey(1)
    if key==27:
        should_continue = False
        cv2.destroyAllWindows()
        break
cam.EndAcquisition()

In [ ]:
dir(image_result)

In [ ]:
image_result = cam.GetNextImage()

In [ ]:
frameid = image_result.GetFrameID()

In [ ]:
timestamp = image_result.GetTimeStamp()

In [ ]:
datetime.fromtimestamp(timestamp*1e-9)

In [ ]:
timestamp/319619333

In [ ]:
node = nodemap.GetNode('Root')
# Create category node
node_category = PySpin.CCategoryPtr(node)

# Get and print display name
display_name = node_category.GetDisplayName()

In [ ]:
for node_feature in node_category.GetFeatures():
    print()

In [8]:
def print_device_info(nodemap):
    """
    This function prints the device information of the camera from the transport
    layer; please see NodeMapInfo example for more in-depth comments on printing
    device information from the nodemap.

    :param nodemap: Transport layer device nodemap.
    :type nodemap: INodeMap
    :returns: True if successful, False otherwise.
    :rtype: bool
    """

    print('*** DEVICE INFORMATION ***\n')

    try:
        result = True
        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))

        if PySpin.IsAvailable(node_device_information) and PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if PySpin.IsReadable(node_feature) else 'Node not readable'))

        else:
            print('Device control information not available.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result

In [9]:
res = print_device_info(nodemap)

*** DEVICE INFORMATION ***

DeviceID: 17278795
DeviceSerialNumber: 17278795
DeviceVendorName: Point Grey Research
DeviceModelName: Flea3 FL3-U3-13Y3M
DeviceType: U3V
DeviceDisplayName: Point Grey Research Flea3 FL3-U3-13Y3M
DeviceAccessStatus: ReadWrite
DeviceVersion: FW:v2.7.3.00 FPGA:v2.02
DeviceDriverVersion: PGRUSBCam.sys : 2.7.3.235
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_3300&MI_00\7&2DB08C72&0&0000
DeviceCurrentSpeed: SuperSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1


In [11]:
def print_value_node(node, level):
    """
    Retrieves and prints the display name and value of all node types as value nodes.
    A value node is a general node type that allows for the reading and writing of any node type as a string.

    :param node: Node to get information from.
    :type node: INode
    :param level: Depth to indent output.
    :type level: int
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        # Create value node
        node_value = PySpin.CValuePtr(node)

        # Retrieve display name
        #
        # *** NOTES ***
        # A node's 'display name' is generally more appropriate for output and
        # user interaction whereas its 'name' is what the camera understands.
        # Generally, its name is the same as its display name but without
        # spaces - for instance, the name of the node that houses a camera's
        # serial number is 'DeviceSerialNumber' while its display name is
        # 'Device Serial Number'.
        display_name = node_value.GetDisplayName()

        # Retrieve value of any node type as string
        #
        # *** NOTES ***
        # Because value nodes return any node type as a string, it can be much
        # easier to deal with nodes as value nodes rather than their actual
        # individual types.
        value = node_value.ToString()

        # Cap length at MAX_CHARS
        value = value[:MAX_CHARS] + '...' if len(value) > MAX_CHARS else value

        # Print value
        print_with_indent(level, '%s: %s' % (display_name, value))

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result

def print_with_indent(level, text):
    """
    Helper function for printing a string prefix with a specifc number of indents.
    :param level: Number of indents to generate
    :type level: int
    :param text: String to print after indent
    :type text: str
    """
    ind = ''
    for i in range(level):
        ind += '    '
    print('%s%s' % (ind, text))
    
def print_category_node_and_all_features(node, level):
    """
    This function retrieves and prints out the display name of a category node
    before printing all child nodes. Child nodes that are also category nodes are
    printed recursively.

    :param node: Category node to get information from.
    :type node: INode
    :param level: Depth to indent output.
    :type level: int
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        # Create category node
        node_category = PySpin.CCategoryPtr(node)

        # Get and print display name
        display_name = node_category.GetDisplayName()
        print_with_indent(level, display_name)

        # Retrieve and iterate through all children
        #
        # *** NOTES ***
        # The two nodes that typically have children are category nodes and
        # enumeration nodes. Throughout the examples, the children of category nodes
        # are referred to as features while the children of enumeration nodes are
        # referred to as entries. Keep in mind that enumeration nodes can be cast as
        # category nodes, but category nodes cannot be cast as enumerations.
        for node_feature in node_category.GetFeatures():

            # Ensure node is available and readable
            if not PySpin.IsAvailable(node_feature) or not PySpin.IsReadable(node_feature):
                continue
            
            # Category nodes must be dealt with separately in order to retrieve subnodes recursively.
            if node_feature.GetPrincipalInterfaceType() == PySpin.intfICategory:
                result &= print_category_node_and_all_features(node_feature, level + 1)

            # Cast all non-category nodes as value nodes
            #
            # *** NOTES ***
            # If dealing with a variety of node types and their values, it may be
            # simpler to cast them as value nodes rather than as their individual types.
            # However, with this increased ease-of-use, functionality is sacrificed.
            elif CHOSEN_READ == ReadType.VALUE:
                result &= print_value_node(node_feature, level + 1)

            # Cast all non-category nodes as actual types
            elif CHOSEN_READ == ReadType.INDIVIDUAL:
                if node_feature.GetPrincipalInterfaceType() == PySpin.intfIString:
                    result &= print_string_node(node_feature, level + 1)
                elif node_feature.GetPrincipalInterfaceType() == PySpin.intfIInteger:
                    result &= print_integer_node(node_feature, level + 1)
                elif node_feature.GetPrincipalInterfaceType() == PySpin.intfIFloat:
                    result &= print_float_node(node_feature, level + 1)
                elif node_feature.GetPrincipalInterfaceType() == PySpin.intfIBoolean:
                    result &= print_boolean_node(node_feature, level + 1)
                elif node_feature.GetPrincipalInterfaceType() == PySpin.intfICommand:
                    result &= print_command_node(node_feature, level + 1)
                elif node_feature.GetPrincipalInterfaceType() == PySpin.intfIEnumeration:
                    result &= print_enumeration_node_and_current_entry(node_feature, level + 1)

        print('')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result

In [12]:
class ReadType:
    """
    Use the following constants to determine whether nodes are read
    as Value nodes or their individual types.
    """
    VALUE = 0,
    INDIVIDUAL = 1

CHOSEN_READ = ReadType.VALUE

In [13]:
MAX_CHARS = 72
level = 0

cam.Init()
nodemap = cam.GetNodeMap()

node = nodemap.GetNode('Root')

# res = print_value_node(node, level)

# Create category node
node_category = PySpin.CCategoryPtr(node)

# Get and print display name
display_name = node_category.GetDisplayName()
print_with_indent(level, display_name)

result = True

for node_feature in node_category.GetFeatures():
    #     if node_feature.GetPrincipalInterfaceType() == PySpin.intfICategory:
    #         result &= print_category_node_and_all_features(node_feature, level + 1)
    #     else:
    #         result &= print_value_node(node_feature, level + 1)
    # result &= print_value_node(node_feature, level+1)
    node_value = PySpin.CValuePtr(node_feature)
    name = node_value.GetName()
    print(name)

Root
AnalogControl
DeviceControl
AcquisitionControl
ImageFormatControl
UserSetControl
DigitalIOControl
TransportLayerControl
ChunkDataControl


In [14]:
for node_feature in node_category.GetFeatures():
    node_value = PySpin.CValuePtr(node_feature)
    name = node_value.GetName()
    if name == 'AcquisitionControl':
        control_node = node_value
        control_feature = node_feature

In [15]:
node_category = PySpin.CCategoryPtr(control_feature)

In [16]:
for node_feature in node_category.GetFeatures():
    node_value = PySpin.CValuePtr(node_feature)
    name = node_value.GetName()
    print(name)
    #     if name =='BinningHorizontal':
    #         control_node = node_value
    #         control_feature = node_feature
    # result &= print_value_node(node_feature, level + 1)

TriggerSelector
TriggerMode
TriggerSoftware
TriggerSource
TriggerActivation
TriggerDelay
TriggerDelayEnabled
ExposureMode
ExposureAuto
ExposureTime
ExposureTimeAbs
AutoExposureTimeLowerLimit
AutoExposureTimeUpperLimit
AcquisitionMode
AcquisitionStart
AcquisitionStop
AcquisitionFrameRateAuto
AcquisitionFrameRateEnabled
AcquisitionFrameRate
AcquisitionFrameCount


In [76]:
def set_value(nodemap, nodename, value):
    try:
        node = nodemap.GetNode(nodename)
        nodetype = node.GetPrincipalInterfaceType()

        nodeval, typestring = get_nodeval_and_type(node)

        assert(PySpin.IsWritable(nodeval))

        if typestring == 'int' or typestring == 'float':
            assert(value <= nodeval.GetMax() and value >= nodeval.GetMin())
        if typestring == 'int':
            assert(type(value)==int)
            if PySpin.IsAvailable(nodeval) and PySpin.IsWritable(nodeval):
                nodeval.SetValue(value)
            else:
                raise ValueError('Node not writable or available: %s' %nodename)

        elif typestring == 'float':
            assert(type(value)==float)
            if PySpin.IsAvailable(nodeval) and PySpin.IsWritable(nodeval):
                nodeval.SetValue(value)
            else:
                raise ValueError('Node not writable or available: %s' %nodename)
        elif typestring == 'enum':
            assert(type(value)==str)

            entry = nodeval.GetEntryByName(value)

            if entry is None:
                print('Valid entries: ')
                entrylist = nodeval.GetEntries()
                for entry in entrylist:
                    print(entry.GetName())
                raise ValueError('Invalid entry!: %s' %value)
            else:
                entry = PySpin.CEnumEntryPtr(entry)
            if PySpin.IsAvailable(entry) and PySpin.IsReadable(entry):
                nodeval.SetIntValue(entry.GetValue())
            else:
                raise ValueError('Entry not readable!')
            # PySpin.CEnumEntryPtr
        elif typestring == 'bool':
            assert(type(value)==bool)
            if PySpin.IsAvailable(nodeval) and PySpin.IsWritable(nodeval):
                nodeval.SetValue(value)
            else:
                raise ValueError('Node not writable or available: %s' %nodename)
    except PySpin.SpinnakerException as e:
        raise ValueError('Error: %s' %e)

In [74]:
def turn_strobe_on(nodemap, line, strobe_duration=0.0):
    assert(type(line)==int)
    assert(type(strobe_duration)==float)
    
    linestr = 'Line%d'%line
    
    # set the line selector to this line so that we change the following
    # values for Line2, for example, not Line0
    set_value(nodemap, 'LineSelector', linestr)
    # one of input, trigger, strobe, output
    set_value(nodemap, 'LineMode', 'Strobe')
    # enable strobe
    set_value(nodemap, 'StrobeEnabled', True)
    # set duration, in us I think?
    set_value(nodemap, 'StrobeDuration', strobe_duration)
    # inverted means low by default, high when strobe is on
    set_value(nodemap, 'LineInverter', True)

In [90]:
def print_value(nodemap, nodename):
    assert(type(nodename)==str)
    node = nodemap.GetNode(nodename)
    nodeval, typestring = get_nodeval_and_type(node)
    if typestring == 'enum':
        # GetCurrentEntry
        print(nodename, typestring, nodeval.ToString())
    else:
        print(nodename, typestring, nodeval.GetValue())

In [95]:
print_value(nodemap, 'BinningVertical')

BinningVertical int 1


In [75]:
def get_nodeval_and_type(node):
    nodetype = node.GetPrincipalInterfaceType()
    if nodetype== PySpin.intfIString:
        nodeval = PySpin.CStringPtr(node)
        typestring = 'string'
    elif nodetype== PySpin.intfIInteger:
        nodeval = PySpin.CIntegerPtr(node)
        typestring = 'int'
    elif nodetype== PySpin.intfIFloat:
        nodeval = PySpin.CFloatPtr(node)
        typestring = 'float'
    elif nodetype== PySpin.intfIBoolean:
        nodeval = PySpin.CBooleanPtr(node)
        typestring = 'bool'
    elif nodetype == PySpin.intfIEnumeration:
        nodeval = PySpin.CEnumerationPtr(node)
        typestring = 'enum'
    elif nodetype == PySpin.intfICommand:
        nodeval = PySpin.CCommandPtr(node)
        typestring = 'command'
    else:
        raise ValueError('Invalid node type: %s' %nodetype)
        
    return(nodeval, typestring)

In [97]:
set_value(nodemap, 'AcquisitionMode', 'Continuous')
set_value(nodemap, 'ExposureAuto', 'Off')
# note, can't change horizontal, but changing vertical
# changes horizontal automatically
set_value(nodemap, 'BinningVertical', 2)

set_value(nodemap, 'Height', 512)
set_value(nodemap, 'Width', 640)
set_value(nodemap, 'OffsetX', 0)
set_value(nodemap, 'OffsetY', 0)
set_value(nodemap, 'PixelFormat', 'Mono8')
set_value(nodemap, 'AcquisitionFrameRate', 60.0)

set_value(nodemap, 'ExposureTime', 1*1000.0) #  in us
set_value(nodemap, 'GainAuto', 'Off') 
set_value(nodemap, 'Gain', 10.0)

set_value(nodemap, 'SharpnessAuto', 'Off') 

In [ ]:
cam.DeInit()

# Clear camera list before releasing system
cam_list.Clear()

# Release system instance
system.ReleaseInstance()